# Fiftyville
This noteboook aims at solving the fiftyville problem using sqlite3 queries. We have a database with information of Fiftyville - a city where there was a crime. We were given the information that the crime occurred on July 28 of 2021.

In [23]:
import sqlite3

## Get the table

In [24]:
def get_table():
    conn = sqlite3.connect('fiftyville.db')
    c = conn.cursor()
    c.execute('SELECT name FROM sqlite_master WHERE type="table"')
    return c


In [25]:
cursor = get_table()

## Tables

In [26]:
# Query to get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

with open('table_fields.txt', 'w') as file:
    # Loop through all tables and write their fields to the file
    for table in tables:
        table_name = table[0]
        file.write(f"Table: {table_name}\n")
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        for column in columns:
            file.write(f"  Column: {column[1]}, Type: {column[2]}\n")
        file.write("\n")

## Crime scene reports
We were given the information that the crime occurred on June 28 2021 in Humphrey Street. We are going to list all crime scene reports on that day.

In [27]:
cursor.execute("SELECT description FROM crime_scene_reports WHERE day = 28 AND month = 7 AND year = 2021 AND street = 'Humphrey Street';")
reports = cursor.fetchall()
for report in reports:
    print(report)

('Theft of the CS50 duck took place at 10:15am at the Humphrey Street bakery. Interviews were conducted today with three witnesses who were present at the time – each of their interview transcripts mentions the bakery.',)
('Littering took place at 16:36. No known witnesses.',)


Now we knwo the time of the theft - `10:15`. We are going to collect more information from the interview transcripts, particularly the ones with `bakery` on it.

In [28]:
cursor.execute("SELECT transcript FROM interviews WHERE day = 28 AND month = 7 AND year = 2021 AND transcript LIKE '%bakery%'")
transcripts = cursor.fetchall()
for item in transcripts:
    print(item)

('Sometime within ten minutes of the theft, I saw the thief get into a car in the bakery parking lot and drive away. If you have security footage from the bakery parking lot, you might want to look for cars that left the parking lot in that time frame.',)
("I don't know the thief's name, but it was someone I recognized. Earlier this morning, before I arrived at Emma's bakery, I was walking by the ATM on Leggett Street and saw the thief there withdrawing some money.",)
('As the thief was leaving the bakery, they called someone who talked to them for less than a minute. In the call, I heard the thief say that they were planning to take the earliest flight out of Fiftyville tomorrow. The thief then asked the person on the other end of the phone to purchase the flight ticket.',)


Now we know 3 more pieces of information:
* The theft's car is on the `bakery_security_logs` as an exit between __10:15__ and __10:25__.
* In the `phone_calls` table the thief's number is registered as the caller, the duration of the phone call is less than __60 seconds__ and it was on the theft's day.
* There was a withdrawal transaction in the ATM in __Leggett Street__ earlier that day whose account number matches the thief's account number.
* The accomplice bought a ticket for the earliest flight out of Fiftyville for the next day.

In [29]:
cursor.execute("SELECT caller_people.name AS caller_name, receiver_people.name AS receiver_name, phone_calls.id FROM \
               phone_calls INNER JOIN \
               people AS caller_people ON phone_calls.caller = caller_people.phone_number INNER JOIN \
               people AS receiver_people ON phone_calls.receiver = receiver_people.phone_number INNER JOIN \
               bakery_security_logs ON bakery_security_logs.license_plate = caller_people.license_plate INNER JOIN \
               bank_accounts ON bank_accounts.person_id = caller_people.id INNER JOIN \
               atm_transactions ON atm_transactions.account_number = bank_accounts.account_number WHERE \
               bakery_security_logs.activity = 'exit' AND bakery_security_logs.day = 28 AND bakery_security_logs.month = 7 AND bakery_security_logs.year = 2021 AND \
               bakery_security_logs.hour = 10 AND bakery_security_logs.minute BETWEEN 15 AND 25 AND \
               phone_calls.duration < 60 AND phone_calls.day = 28 AND phone_calls.month = 7 AND phone_calls.year = 2021 AND \
               atm_transactions.atm_location = 'Leggett Street' AND atm_transactions.transaction_type = 'withdraw' AND \
               atm_transactions.day = 28 AND atm_transactions.month = 7 AND atm_transactions.year = 2021;\
               ")

calls = cursor.fetchall()
for call in calls:
    print(f"Caller: {call[0]}, Receiver: {call[1]}, Call ID: {call[2]}")



Caller: Bruce, Receiver: Robin, Call ID: 233
Caller: Diana, Receiver: Philip, Call ID: 255


Now we have 2 possible thief-accomplice options. We will get information about the earliest flight out of Fiftyville on July 29th 2021 and who took that flight.

In [30]:
cursor.execute("SELECT * FROM flights WHERE flights.day = 29 AND flights.month = 7 AND flights.year = 2021;")
flights = cursor.fetchall()
for flight in flights:
    print(flight)

(18, 8, 6, 2021, 7, 29, 16, 0)
(23, 8, 11, 2021, 7, 29, 12, 15)
(36, 8, 4, 2021, 7, 29, 8, 20)
(43, 8, 1, 2021, 7, 29, 9, 30)
(53, 8, 9, 2021, 7, 29, 15, 20)


The earliest flight in this list has `id = 36`. We will see which city the flight heads to.

In [31]:
cursor.execute("SELECT airports.city FROM airports INNER JOIN flights ON flights.destination_airport_id = airports.id \
               WHERE flights.id = 36; ")
destinations = cursor.fetchall()
for d in destinations:
    print(d)

('New York City',)


In [32]:
cursor.execute("SELECT people.name FROM passengers INNER JOIN people ON passengers.passport_number = people.passport_number WHERE \
               passengers.flight_id = 36 AND (people.name = 'Bruce' OR people.name = 'Diana');")
passengers = cursor.fetchall()
for passenger in passengers:
    print(passenger)

('Bruce',)


Given that only Bruce escaped from Fiftyville, we conclude that:
* `Bruce` is the thief.
* `Robin` is the accomplice.
* The thief escaped to `New York City`.
